In [40]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [ ]:
import random
import glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.patches as patches
import pandas as pd
import seaborn as sns
import math 
import time
import os
import sys
from ctypes import *
from math import radians, cos, sin, asin, sqrt
%matplotlib inline
# %matplotlib nbagg
# import mpld3
# mpld3.enable_notebook()

#Find distance between two lan:lon points in meters
def meters(lat1, lon1, lat2, lon2):  
    R = 6378.137 # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));
    d = R * c
    return d * 1000 # meters

def populate_staypts_df(file_src):
    global staypts_df
    list_of_dfs= []
    filenames = glob.glob(file_src)
    
    #Read the files
    for filename in filenames:
        if os.path.getsize(filename) > 0:
            list_of_dfs.append(pd.read_csv(filename, skiprows=2, sep='\t', header = None))
#     list_of_dfs = [pd.read_csv(filename, skiprows=2, sep='\t', header = None) for filename in filenames]
    
    #put the data from list into one dataframe
    staypts_df = pd.concat(list_of_dfs, ignore_index=True)
    
    staypts_df.columns = ['Latitude', 'Longitude', 'ArrivalTime', 'LeavingTime', 'ClusterId']
    staypts_df['ArrivalTime']= staypts_df['ArrivalTime'].str.replace(","," ")
    staypts_df['LeavingTime']= staypts_df['LeavingTime'].str.replace(","," ")
    staypts_df.ArrivalTime = pd.to_datetime(staypts_df.ArrivalTime)
    staypts_df.LeavingTime = pd.to_datetime(staypts_df.LeavingTime)
    staypts_df = staypts_df.sort_values(['ArrivalTime'])
    staypts_df = staypts_df.reset_index(drop=True)
    
    staypts_df['ClusterId'] = staypts_df.index
    staypts_df['ClusterMeanLat'] = staypts_df['Latitude']
    staypts_df['ClusterMeanLon'] = staypts_df['Longitude']
    
    
    
def cluster_staypts(dest_file):
    global staypts_df
    
    #this fucntion groups the clusters together from different days 
    #Copy the stay points dataframe into another dataframe and remove duplicates
    staypts_df1 = staypts_df[['ClusterId', 'ClusterMeanLat', 'ClusterMeanLon']].copy()
    staypts_df1 = staypts_df1.drop_duplicates(subset=['ClusterId', 'ClusterMeanLat', 'ClusterMeanLon'])

    staypts_df1 = staypts_df1.sort_values(['ClusterMeanLat', 'ClusterMeanLon'])
    staypts_df1 = staypts_df1.reset_index(drop=True)
    
    row = 1
    for i in range(0, len(staypts_df1)):
        for j in range(i+1, len(staypts_df1)):
            #import pdb; pdb.set_trace()
        
            chk_cluster = staypts_df1['ClusterId'][i]
            chk_clulat = staypts_df1['ClusterMeanLat'][i]
            chk_clulon = staypts_df1['ClusterMeanLon'][i]
            curr_cluster = staypts_df1['ClusterId'][j]
            curr_clulat = staypts_df1['ClusterMeanLat'][j]
            curr_clulon = staypts_df1['ClusterMeanLon'][j]
        
            if meters(chk_clulat, chk_clulon, curr_clulat, curr_clulon)<= thersh_dist:
                staypts_df.loc[ (staypts_df['ClusterId']==curr_cluster), 'ClusterId'] = chk_cluster
                staypts_df['ClusterMeanLat'] = staypts_df.groupby('ClusterId')['ClusterMeanLat'].transform(np.mean)
                staypts_df['ClusterMeanLon'] = staypts_df.groupby('ClusterId')['ClusterMeanLon'].transform(np.mean)
            else:
                break
                
    staypts_df.to_csv(dest_file, sep='\t', encoding='utf-8')
    
    
def visualize(dest_file):
    
    dicts = {}
    clu_list = []
    clu_list = staypts_df['ClusterId'].unique()

    #assign for each cluster id a color
    r = lambda: random.randint(0,255)
    for i in range(0, len(clu_list)):
        dicts[clu_list[i]] = ('#%02X%02X%02X' % (r(),r(),r()))

    #create a new graph where we will later add rectangles for each hour:cluster
    fig2 = plt.figure()
    ax1 = fig2.add_subplot(111, aspect='equal')
    fig2.set_size_inches(60, 60)
    #get all the dates for y axis
    date_list = staypts_df['ArrivalTime'].dt.date.unique()
    hour_list = list(range(25))
    x = range(0, 25)
    def_xticks = hour_list
    plt.xticks(x, def_xticks)
    y = range(0, len(date_list)+1)
    def_yticks = date_list
    plt.yticks(y, def_yticks)

    #set the x axis limit from 0-24 hours of a day, y axis with dates
    limsx = (0, 25)
    limsy = (0, len(date_list) +1)

    date_counter = 0
    last_date = staypts_df['ArrivalTime'][0].date()

    for i in range(0, 25):
        ax1.axvline(x= i, linewidth=1, color='r')

    for i in range(0, len(staypts_df)):
        #import pdb; pdb.set_trace()

        if (last_date != staypts_df['ArrivalTime'][i].date()):
                date_counter = date_counter + 1
                ax1.axhline(y= date_counter, linewidth=1, color='r')
                last_date = staypts_df['ArrivalTime'][i].date()

        #if the staypoint stays in the same day
        if (staypts_df['ArrivalTime'][i].date() == staypts_df['LeavingTime'][i].date()):
            a = staypts_df['ArrivalTime'][i].hour + staypts_df['ArrivalTime'][i].minute/60
            b = staypts_df['LeavingTime'][i].hour + staypts_df['LeavingTime'][i].minute/60
            width = b - a
            height = 1

            col_id = dicts.get(staypts_df['ClusterId'][i])
            ax1.add_patch(patches.Rectangle((a, date_counter), width, height, color=col_id, label=staypts_df['ClusterId'][i]))
        #if the staypoint leaves the day
        else:
            #import pdb; pdb.set_trace()
            #patch 1
            a = staypts_df['ArrivalTime'][i].hour + staypts_df['ArrivalTime'][i].minute/60
            b = 24
            width = b - a
            height = 1

            col_id = dicts.get(staypts_df['ClusterId'][i])
            ax1.add_patch(patches.Rectangle((a, date_counter), width, height, color=col_id, label=staypts_df['ClusterId'][i]))
            #patch 2
            a = 0
            b = staypts_df['LeavingTime'][i].hour + staypts_df['LeavingTime'][i].minute/60
            width = b - a
            height = 1

            col_id = dicts.get(staypts_df['ClusterId'][i])
            ax1.add_patch(patches.Rectangle((a, date_counter + 1), width, height, color=col_id, label=staypts_df['ClusterId'][i]))

    plt.savefig(dest_file)

#     handles, labels = ax1.get_legend_handles_labels()
#     handle_list, label_list = [], []
#     for handle, label in zip(handles, labels):
#         if label not in label_list:
#             handle_list.append(handle)
#             label_list.append(label)
#     plt.legend(handle_list, label_list)

#     plt.xlim(limsx)
#     plt.ylim(limsy)
#------------------------------------------------------------
def main():
    for i in range(6, 51):
        user = "%03d"%i
        file_src = file_path + str(user) + '/Trajectory/20*.plt'

        populate_staypts_df(file_src)
        
        dest_file = destination + str(user) + " all.csv" 
        cluster_staypts(dest_file)
        
        dest_file = destination + str(user) + " vis.png"
        visualize(dest_file)

#---------------------------------------------------------------
thersh_dist = 200
time_format = '%H:%M:%S'
staypts_df = pd.DataFrame()
file_path = '/home/shashank/Documents/location/code/several tests/test 4/results/StayPoint/'
destination = '/home/shashank/Documents/location/code/several tests/test 4/results/staypoints vis/'

main()